In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import torch
from transformers import AutoTokenizer, AutoModel
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dropout, Dense,Embedding
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, Dataset

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/thang/data1.csv", sep=";",names=["sent", "sentiment"])
df= df.sample(n=50000, random_state=42)

In [ ]:
# Chia tập train (80%) và tập còn lại (20%)
train_df, temp_df = train_test_split(df, test_size=0.2, random_state=42, stratify=df["sentiment"])

# Chia tiếp tập còn lại (20%) thành validation (10%) và test (10%)
valid_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42, stratify=temp_df["sentiment"])

# Kiểm tra kích thước từng tập
print(f"Train size: {len(train_df)}")
print(f"Validation size: {len(valid_df)}")
print(f"Test size: {len(test_df)}")
possible_sentiments = df.sentiment.unique()

sentiment_dict = {}
for index, possible_sentiments in enumerate(possible_sentiments):
    sentiment_dict[possible_sentiments] = index

print(sentiment_dict)

train_df['label'] = df.sentiment.replace(sentiment_dict)
df.head(20)

valid_df['label'] = valid_df.sentiment.replace(sentiment_dict)
valid_df.head(20)

test_df['label'] = test_df.sentiment.replace(sentiment_dict)
test_df.head(20)

Train size: 40000
Validation size: 5000
Test size: 5000
{'positive': 0, 'negative': 1}


<ipython-input-4-68b421a7cd48>:19: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  train_df['label'] = df.sentiment.replace(sentiment_dict)
<ipython-input-4-68b421a7cd48>:22: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  valid_df['label'] = valid_df.sentiment.replace(sentiment_dict)
<ipython-input-4-68b421a7cd48>:25: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, s

,sent,sentiment,label
29304,"phim làm rất tệ, các cảnh hành động/bạo lực lố...",negative,1
11033,Zentropa có nhiều điểm chung với Người đàn ông...,positive,0
3430,Thật sai lầm khi coi bất kỳ bộ phim nào của th...,positive,0
42021,"Tiền đề rất hấp dẫn, nhưng thật đáng buồn là b...",negative,1
5792,Barney không dạy bọn trẻ gì cả!!! Dưới đây là ...,negative,1
22118,*Cảnh báo: 1 spoiler nhỏ không quan trọng* Bạn...,negative,1
43133,Born Again là một tập dưới tiêu chuẩn từ phần ...,negative,1
27405,Mulva bị Teen Ape đưa vào tình trạng sugercoma...,negative,1
45378,"Được rồi, tôi đã đánh dấu phần spoiler này nên...",positive,0
4284,"Vậy đây là ""Phim dành cho truyền hình"" của HBO...",negative,1


In [ ]:
X_train = pd.DataFrame(train_df, columns=['sent'])
X_dev = pd.DataFrame(valid_df, columns=['sent'])
X_test = pd.DataFrame(test_df, columns=['sent'])
y_train = pd.DataFrame(train_df, columns=['label'])
y_dev = pd.DataFrame(valid_df, columns=['label'])
y_test = pd.DataFrame(test_df, columns=['label'])

In [ ]:


tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base")
phobert = AutoModel.from_pretrained("vinai/phobert-base").to("cuda")

def get_phobert_embeddings(texts, batch_size=256):
    phobert.eval()
    all_embeddings = []

    # Tokenize toàn bộ dữ liệu
    encodings = tokenizer(texts, return_tensors="pt", truncation=True, padding=True, max_length=256)

    # Tạo DataLoader để batch xử lý
    dataset = torch.utils.data.TensorDataset(encodings["input_ids"], encodings["attention_mask"])
    dataloader = DataLoader(dataset, batch_size=batch_size)

    total_samples = len(texts)
    processed_samples = 0

    with torch.no_grad():
        for batch in dataloader:
            input_ids, attention_mask = [t.to("cuda") for t in batch]
            outputs = phobert(input_ids, attention_mask=attention_mask)
            batch_embeddings = outputs.last_hidden_state.mean(dim=1)
            all_embeddings.append(batch_embeddings.cpu().numpy())

            # Cập nhật số lượng mẫu đã xử lý
            processed_samples += len(input_ids)
            print(f"Processed {processed_samples}/{total_samples} samples...")

    return np.vstack(all_embeddings)

# Chạy và hiển thị tiến trình
X_train = get_phobert_embeddings(X_train["sent"].tolist())
X_dev = get_phobert_embeddings(X_dev["sent"].tolist())
X_test = get_phobert_embeddings(X_test["sent"].tolist())

print("Shape of X_train:", X_train.shape)
print("Shape of X_dev:", X_dev.shape)
print("Shape of X_test", X_test.shape)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/557 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/895k [00:00<?, ?B/s]

bpe.codes:   0%|          | 0.00/1.14M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.13M [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/543M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/543M [00:00<?, ?B/s]

Processed 256/40000 samples...
Processed 512/40000 samples...
Processed 768/40000 samples...
Processed 1024/40000 samples...
Processed 1280/40000 samples...
Processed 1536/40000 samples...
Processed 1792/40000 samples...
Processed 2048/40000 samples...
Processed 2304/40000 samples...
Processed 2560/40000 samples...
Processed 2816/40000 samples...
Processed 3072/40000 samples...
Processed 3328/40000 samples...
Processed 3584/40000 samples...
Processed 3840/40000 samples...
Processed 4096/40000 samples...
Processed 4352/40000 samples...
Processed 4608/40000 samples...
Processed 4864/40000 samples...
Processed 5120/40000 samples...
Processed 5376/40000 samples...
Processed 5632/40000 samples...
Processed 5888/40000 samples...
Processed 6144/40000 samples...
Processed 6400/40000 samples...
Processed 6656/40000 samples...
Processed 6912/40000 samples...
Processed 7168/40000 samples...
Processed 7424/40000 samples...
Processed 7680/40000 samples...
Processed 7936/40000 samples...
Processed 8

In [ ]:
X_train = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))
X_dev = X_dev.reshape((X_dev.shape[0], 1, X_dev.shape[1]))

In [ ]:

model = Sequential([
    LSTM(128, return_sequences=True, input_shape=(1, X_train.shape[2])),
    Dropout(0.5),
    LSTM(32, return_sequences=False),
    Dense(32, activation='relu'),
    Dropout(0.5),
    Dense(16, activation='relu'),
    Dropout(0.5),
    Dense(3, activation='softmax')
])


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [ ]:
model.compile(optimizer=Adam(), loss='sparse_categorical_crossentropy', metrics=['accuracy'])


In [ ]:
X_train = X_train.astype(float)
y_train = y_train.astype(float)

X_dev = X_dev.astype(float)
y_dev = y_dev.astype(float)


In [ ]:
history = model.fit(
    X_train,  # Dữ liệu huấn luyện
    y_train,  # Nhãn huấn luyện
    epochs=30,  # Số epochs
    batch_size=256,# Kích thước batch
    validation_data=(X_dev, y_dev)  # Dữ liệu validation
)


Epoch 1/30
157/157 ━━━━━━━━━━━━━━━━━━━━ 6s 11ms/step - accuracy: 0.5143 - loss: 0.9052 - val_accuracy: 0.8088 - val_loss: 0.4475
Epoch 2/30
157/157 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - accuracy: 0.7409 - loss: 0.5725 - val_accuracy: 0.8038 - val_loss: 0.4210
Epoch 3/30
157/157 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - accuracy: 0.7831 - loss: 0.5096 - val_accuracy: 0.8290 - val_loss: 0.3792
Epoch 4/30
157/157 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.7991 - loss: 0.4670 - val_accuracy: 0.8266 - val_loss: 0.3777
Epoch 5/30
157/157 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.8055 - loss: 0.4605 - val_accuracy: 0.8254 - val_loss: 0.3864
Epoch 6/30
157/157 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.8154 - loss: 0.4375 - val_accuracy: 0.8228 - val_loss: 0.3855
Epoch 7/30
157/157 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.8178 - loss: 0.4314 - val_accuracy: 0.8254 - val_loss: 0.3735
Epoch 8/30
157/157 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.8195 - loss: 0.4222 - val_accuracy: 

In [ ]:
X_test = X_test.reshape(X_test.shape[0], 1, X_test.shape[1])


In [ ]:
X_test = X_test.astype(float)
y_test = y_test.astype(float)

In [ ]:
# Đánh giá mô hình trên dữ liệu huấn luyện

test_loss, test_accuracy = model.evaluate(X_test, y_test)

# In kết quả
print(f'Training Loss: {test_loss}')
print(f'Training Accuracy: {test_accuracy}')


157/157 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8359 - loss: 0.3711
Training Loss: 0.3765822649002075
Training Accuracy: 0.8299999833106995


In [ ]:
from sklearn.metrics import classification_report
y_pred = model.predict(X_test)
y_pred_labels = np.argmax(y_pred, axis=1)  # Nếu output là xác suất của nhiều lớp
y_pred = model.predict(X_test)
y_pred_labels = np.argmax(y_pred, axis=1)
print(classification_report(y_test, y_pred_labels))

157/157 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step
157/157 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
              precision    recall  f1-score   support

         0.0       0.80      0.88      0.84      2500
         1.0       0.87      0.78      0.82      2500

    accuracy                           0.83      5000
   macro avg       0.83      0.83      0.83      5000
weighted avg       0.83      0.83      0.83      5000



In [ ]:
sample_text = input("Nhập một câu: ")

# Chuyển câu nhập thành embedding
sample_emb = get_phobert_embeddings([sample_text])
# Reshape lại để phù hợp với input của LSTM
sample_emb = sample_emb.reshape( sample_emb.shape[0], 1, sample_emb.shape[1])

# Dự đoán
predictions = model.predict(sample_emb)

# Lấy nhãn có xác suất cao nhất
predicted_label = np.argmax(predictions, axis=1)

# Hiển thị kết quả
print(f"Nhãn dự đoán: {predicted_label[0]}")

Nhập một câu: hay
Processed 1/1 samples...
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
Nhãn dự đoán: 0


In [ ]:
import numpy as np
from sklearn.model_selection import KFold
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dropout, Dense
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import accuracy_score

# Create model function
def create_model(input_shape):
    model = Sequential([
        LSTM(128, return_sequences=True, input_shape=(input_shape)),
    Dropout(0.5),
    LSTM(32, return_sequences=False),
    Dense(32, activation='relu'),
    Dropout(0.5),
    Dense(16, activation='relu'),
    Dropout(0.5),
    Dense(3, activation='softmax')  # Adjust the number of units to match your problem
    ])
    model.compile(optimizer=Adam(), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

kf = KFold(n_splits=5, shuffle=True, random_state=42)

accuracies = []
y_train = np.array(y_train)
for train_index, val_index in kf.split(X_train):
    X_train_fold, X_val_fold = X_train[train_index], X_train[val_index]
    y_train_fold, y_val_fold = y_train[train_index], y_train[val_index]

    model1 = create_model((1, X_train_fold.shape[2]))  # Adjust input shape

    model1.fit(X_train_fold, y_train_fold, epochs=30, batch_size=256, verbose=1)  # You can adjust epochs

    y_pred_fold = model1.predict(X_val_fold)
    y_pred_fold = np.argmax(y_pred_fold, axis=1)  # Convert predictions to class labels

    accuracy = accuracy_score(y_val_fold, y_pred_fold)  # Calculate accuracy
    print(f'Cross-Validation Accuracy: {accuracy}')
    accuracies.append(accuracy)
print(f'mean Cross-Validation Accuracy: {np.mean(accuracies)}')
print(f'max Cross-Validation Accuracy: {np.max(accuracies)}')


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/30
125/125 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - accuracy: 0.4830 - loss: 0.8592
Epoch 2/30
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.5084 - loss: 0.7197
Epoch 3/30
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.6968 - loss: 0.5956
Epoch 4/30
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.7871 - loss: 0.5069
Epoch 5/30
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.7989 - loss: 0.4824
Epoch 6/30
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.8074 - loss: 0.4573
Epoch 7/30
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.8135 - loss: 0.4435
Epoch 8/30
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.8164 - loss: 0.4353
Epoch 9/30
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.8220 - loss: 0.4264
Epoch 10/30
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.8230 - loss: 0.4224
Epoch 11/30
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.8250 - loss: 0.4171
Epoch 12/30
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step

/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/30
125/125 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - accuracy: 0.4641 - loss: 0.9844
Epoch 2/30
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.7209 - loss: 0.6254
Epoch 3/30
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.7877 - loss: 0.5137
Epoch 4/30
125/125 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - accuracy: 0.7972 - loss: 0.4805
Epoch 5/30
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.7995 - loss: 0.4695
Epoch 6/30
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.8068 - loss: 0.4512
Epoch 7/30
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.8134 - loss: 0.4432
Epoch 8/30
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.8138 - loss: 0.4318
Epoch 9/30
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.8163 - loss: 0.4292
Epoch 10/30
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.8188 - loss: 0.4322
Epoch 11/30
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.8207 - loss: 0.4189
Epoch 12/30
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/st

/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/30
125/125 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - accuracy: 0.4795 - loss: 0.9146
Epoch 2/30
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.6985 - loss: 0.5972
Epoch 3/30
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.7825 - loss: 0.4940
Epoch 4/30
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.7985 - loss: 0.4699
Epoch 5/30
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.8071 - loss: 0.4497
Epoch 6/30
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.8105 - loss: 0.4432
Epoch 7/30
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.8206 - loss: 0.4301
Epoch 8/30
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.8171 - loss: 0.4243
Epoch 9/30
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.8212 - loss: 0.4228
Epoch 10/30
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.8245 - loss: 0.4128
Epoch 11/30
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.8259 - loss: 0.4157
Epoch 12/30
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step

/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/30
125/125 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - accuracy: 0.4909 - loss: 0.9487
Epoch 2/30
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.7155 - loss: 0.6008
Epoch 3/30
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.7786 - loss: 0.5093
Epoch 4/30
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.8007 - loss: 0.4744
Epoch 5/30
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.8019 - loss: 0.4621
Epoch 6/30
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.8111 - loss: 0.4449
Epoch 7/30
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.8175 - loss: 0.4314
Epoch 8/30
125/125 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - accuracy: 0.8204 - loss: 0.4345
Epoch 9/30
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.8194 - loss: 0.4266
Epoch 10/30
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.8217 - loss: 0.4213
Epoch 11/30
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.8237 - loss: 0.4139
Epoch 12/30
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/ste

/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/30
125/125 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - accuracy: 0.4812 - loss: 0.9600
Epoch 2/30
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.7156 - loss: 0.6228
Epoch 3/30
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.7865 - loss: 0.5141
Epoch 4/30
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.7981 - loss: 0.4809
Epoch 5/30
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.8084 - loss: 0.4589
Epoch 6/30
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.8117 - loss: 0.4467
Epoch 7/30
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.8082 - loss: 0.4443
Epoch 8/30
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.8147 - loss: 0.4313
Epoch 9/30
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.8122 - loss: 0.4343
Epoch 10/30
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.8232 - loss: 0.4147
Epoch 11/30
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.8199 - loss: 0.4183
Epoch 12/30
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step

In [ ]:
from sklearn.metrics import classification_report
y_pred = model1.predict(X_test)
y_pred_labels = np.argmax(y_pred, axis=1)
print(classification_report(y_test, y_pred_labels))

157/157 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step
              precision    recall  f1-score   support

         0.0       0.80      0.87      0.83      2500
         1.0       0.86      0.78      0.82      2500

    accuracy                           0.83      5000
   macro avg       0.83      0.83      0.83      5000
weighted avg       0.83      0.83      0.83      5000

